In [56]:
#Import Libraries

%matplotlib inline
import seaborn as sns
import numpy as np
import scipy.stats.stats as stats
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf
import matplotlib.dates as mdates
import pandas as pd
import datetime as dt
import os

In [67]:
#Where we store the h5 data
folder = "E:/TwitchData/"
#Some data doesnt exist in the beginning, change incomplete to true if so
incomplete = False
ref_date_folder = "E:/TwitchData/"
data_names = os.listdir(folder)

#Get only the .h5 files
data_names = [x for x in data_names if x[-3:] == ".h5"]
print("Our Files:")
print(data_names)

Our Files:
['Data-AsOf-2018-11-07.h5', 'Data-AsOf-2018-11-08.h5', 'Data-AsOf-2018-11-09.h5', 'Data-AsOf-2018-11-10.h5', 'Data-AsOf-2018-11-11.h5', 'Data-AsOf-2018-11-12.h5', 'Data-AsOf-2018-11-13.h5', 'Data-AsOf-2018-11-14.h5', 'Data-AsOf-2018-11-15.h5', 'Data-AsOf-2018-11-16.h5', 'Data-AsOf-2018-11-17.h5', 'Data-AsOf-2018-11-18.h5', 'Data-AsOf-2018-11-19.h5', 'Data-AsOf-2018-11-20.h5', 'Data-AsOf-2018-11-21.h5', 'Data-AsOf-2018-11-22.h5', 'Data-AsOf-2018-11-23.h5', 'Data-AsOf-2018-11-24.h5', 'Data-AsOf-2018-11-25.h5', 'Data-AsOf-2018-11-26.h5', 'Data-AsOf-2018-11-27.h5', 'Data-AsOf-2018-11-28.h5', 'Data-AsOf-2018-11-29.h5', 'Data-AsOf-2018-11-30.h5', 'Data-AsOf-2018-12-1.h5', 'Data-AsOf-2018-12-2.h5', 'Data-AsOf-2018-12-3.h5']


In [58]:
#Notes down the day series from the first table
#This dataset has a recording for every five minutes
file_name = 'Data-AsOf-2018-11-10.h5'
time = file_name.split("-")[2:]
time = dt.datetime(int(time[0]), int(time[1]), int(time[2][:-3]))
file = pd.HDFStore(ref_date_folder + file_name,"r")
time_of_day = file["Datalist"]["Time"]
file.close()
print(len(time_of_day))

288


In [59]:
game = "Fortnite"

#The following loop notes down the view data for the specified game
total = pd.DataFrame(index = time_of_day)
for file_name in data_names:
    time = file_name.split("-")[2:]
    time = dt.datetime(int(time[0]), int(time[1]), int(time[2][:-3]))
    day_views = []
    time_sig = []
    df = pd.DataFrame()
    #try:
    file = pd.HDFStore(folder + file_name,"r")
    for point in file["Datalist"]["Twitch"]:
        table = file[point]
        day_views = day_views + [int(table[table["Game"] == game].iloc[0]["Views"])]
        time_sig = time_sig + [str(point.split("-")[-2] + "-" + point.split("-")[-1])]
        df = pd.DataFrame(day_views, index = time_sig, columns = [time])
#     except:
#         print(file_name + " has a table that could not be found.")
    total = pd.concat([total,df], axis = 1, sort = False)
    file.close()

print(total.shape)
total.head()

(288, 27)


,2018-11-07 00:00:00,2018-11-08 00:00:00,2018-11-09 00:00:00,2018-11-10 00:00:00,2018-11-11 00:00:00,2018-11-12 00:00:00,2018-11-13 00:00:00,2018-11-14 00:00:00,2018-11-15 00:00:00,2018-11-16 00:00:00,...,2018-11-24 00:00:00,2018-11-25 00:00:00,2018-11-26 00:00:00,2018-11-27 00:00:00,2018-11-28 00:00:00,2018-11-29 00:00:00,2018-11-30 00:00:00,2018-12-01 00:00:00,2018-12-02 00:00:00,2018-12-03 00:00:00
0-0,207994.0,226625.0,269941.0,220553,177618,247830.0,166692.0,159119.0,183081.0,148658.0,...,181099,298735,303835.0,166109.0,200998.0,204734.0,190229.0,215377,235252.0,262627.0
0-5,216181.0,236422.0,275043.0,229993,185546,252608.0,171966.0,162429.0,186184.0,158494.0,...,184211,296049,296525.0,166548.0,208067.0,208998.0,207031.0,214659,240321.0,266235.0
0-10,211861.0,229388.0,253042.0,228710,171763,249824.0,169374.0,155633.0,177825.0,164955.0,...,181193,293967,304513.0,163801.0,204596.0,203712.0,206999.0,205733,235915.0,260402.0
0-15,209152.0,226738.0,249877.0,226396,161097,234340.0,161019.0,158266.0,178856.0,159691.0,...,185146,302725,298230.0,155461.0,199786.0,201540.0,206635.0,200158,232562.0,255791.0
0-20,214155.0,226615.0,257532.0,220971,159547,230043.0,160950.0,154970.0,182454.0,161320.0,...,201252,294327,290636.0,156251.0,203215.0,200610.0,209835.0,182988,228667.0,250071.0


In [60]:
#fills in lost data
total = total.bfill().ffill()
total.tail()

,2018-11-07 00:00:00,2018-11-08 00:00:00,2018-11-09 00:00:00,2018-11-10 00:00:00,2018-11-11 00:00:00,2018-11-12 00:00:00,2018-11-13 00:00:00,2018-11-14 00:00:00,2018-11-15 00:00:00,2018-11-16 00:00:00,...,2018-11-24 00:00:00,2018-11-25 00:00:00,2018-11-26 00:00:00,2018-11-27 00:00:00,2018-11-28 00:00:00,2018-11-29 00:00:00,2018-11-30 00:00:00,2018-12-01 00:00:00,2018-12-02 00:00:00,2018-12-03 00:00:00
23-35,235895.0,252673.0,226696.0,164731,259957,158136.0,177749.0,184123.0,142957.0,163971.0,...,304519,292540,166680.0,195538.0,185808.0,166081.0,207962.0,223665,245892.0,199600.0
23-40,237723.0,257824.0,222566.0,167390,257225,163585.0,166960.0,185094.0,143099.0,166308.0,...,304858,291117,163168.0,197997.0,188019.0,170251.0,210808.0,219146,252332.0,205301.0
23-45,230467.0,265315.0,221928.0,166844,260775,160845.0,167513.0,180727.0,142752.0,166267.0,...,303257,292271,162047.0,195746.0,193209.0,180310.0,212253.0,217099,256836.0,211831.0
23-50,229519.0,264635.0,224463.0,168163,251258,165036.0,169777.0,180874.0,144037.0,204669.0,...,301433,295876,162135.0,195941.0,198643.0,183686.0,213504.0,221571,265417.0,197544.0
23-55,221752.0,267261.0,219691.0,172137,248690,161938.0,156940.0,182330.0,148471.0,207593.0,...,299987,298128,163468.0,196994.0,210214.0,188046.0,213037.0,231281,265269.0,200657.0


In [61]:
#Our return function
def get_ret(a, b):
    return round(float((np.log(a)-np.log(b))*100.0), 5)

In [62]:
#Calculates the return table from view table

return_total = total.copy()
col_max = total.shape[1]
row_max = total.shape[0]
for col, col_num in zip(total.columns, range(col_max)):
    for row, row_num in zip(total.index, range(row_max)):
        if col == total.columns[0] and row == total.index[0]:
            return_total.at[total.index[0],total.columns[0]] = 0
        elif row == total.index[0]:
            return_total.at[row,col] = round(get_ret(total.at[row,col], total.at[total.index[-1], total.columns[col_num-1]]),5)
        else:
            return_total.at[row,col] = round(get_ret(total.at[row,col], total.at[total.index[row_num-1], col]),5)

In [63]:
return_total

,2018-11-07 00:00:00,2018-11-08 00:00:00,2018-11-09 00:00:00,2018-11-10 00:00:00,2018-11-11 00:00:00,2018-11-12 00:00:00,2018-11-13 00:00:00,2018-11-14 00:00:00,2018-11-15 00:00:00,2018-11-16 00:00:00,...,2018-11-24 00:00:00,2018-11-25 00:00:00,2018-11-26 00:00:00,2018-11-27 00:00:00,2018-11-28 00:00:00,2018-11-29 00:00:00,2018-11-30 00:00:00,2018-12-01 00:00:00,2018-12-02 00:00:00,2018-12-03 00:00:00
0-0,0.00000,2.17370,0.99777,0,3,-0.34641,2.89343,1.37888,0.41104,0.12587,...,3,0,1.89619,1.60269,2.01217,-2.64145,1.15420,1,1.70239,-1.00096
0-5,3.86068,4.23217,1.87240,4,4,1.90959,3.11490,2.05886,1.68068,6.40684,...,1,0,-2.43533,0.26394,3.45652,2.06131,8.46399,0,2.13182,1.36446
0-10,-2.01856,-3.02035,-8.33720,0,-7,-1.10822,-1.51875,-4.27403,-4.59355,3.99560,...,-1,0,2.65822,-1.66313,-1.68228,-2.56175,-0.01546,-4,-1.85040,-2.21528
0-15,-1.28691,-1.16197,-1.25867,-1,-6,-6.39836,-5.05869,1.67765,0.57811,-3.24320,...,2,2,-2.08488,-5.22574,-2.37905,-1.07194,-0.17600,-2,-1.43147,-1.78659
0-20,2.36388,-0.05426,3.01752,-2,0,-1.85068,-0.04286,-2.10456,1.99171,1.01493,...,8,-2,-2.57934,0.50688,1.70177,-0.46251,1.53676,-8,-1.68901,-2.26158
0-25,0.76891,0.13935,-3.45549,-2,0,-1.15821,-0.01056,1.39501,1.43819,1.25052,...,-10,2,0.81419,5.86670,0.69145,0.70139,-7.52212,2,5.94439,1.84279
0-30,4.17646,0.03701,-13.49099,1,0,0.00792,-0.90132,-0.92056,0.71812,-0.69109,...,0,-2,-1.05740,-9.04894,2.17914,0.88606,-4.17740,5,4.18131,2.98802
0-35,-6.22077,1.75964,5.68913,-19,-7,-0.94370,-0.36495,2.41306,2.53791,-4.30590,...,2,-25,-6.50220,-0.22025,-0.37703,1.48727,-9.48581,-3,-1.62276,-4.47873
0-40,0.31496,-25.41993,-4.23917,0,-1,-0.82371,2.15030,-0.65098,-4.65203,-0.74997,...,-4,5,-1.39851,0.00000,-2.58169,0.62355,1.89275,1,-4.67624,-0.14675
0-45,0.22275,2.51357,-0.57838,0,3,0.06309,2.85665,-1.58755,-6.62804,1.81705,...,2,0,0.45212,-0.16501,2.00406,-4.89477,0.41182,-6,3.89516,-1.51105


In [66]:
#transforms data to a single file table for easier manipulation
single_file = pd.DataFrame(columns = [game])
for col in return_total.columns:
    app = pd.DataFrame(list(return_total[col]), columns = [game], index = time_of_day)
    single_file = single_file.append(app)
single_file.tail()

,Fortnite
23-35,3.67752
23-40,2.81618
23-45,3.13116
23-50,-6.98274
23-55,1.56356


In [68]:
# Output file

#Deletes leading 0s (Incomplete files backfilled them)
if incomplete:
    for index in single_file.index:
        if(single_file.at[index, game][0] == 0):
            single_file = single_file.drop([index])
        else:
            break
        
single_file.to_excel(game + "_ret.xlsx")